# One Hot Encoding and Feature engineering

In this notebook I will:
+ Split the categorical columns out into One Hot encoded columns


### Contents:
- [Data Import](#Data-Import)
- [One hot coding](#One-Hot-Coding)

# Data Import

In [1595]:
# load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV,ElasticNet, ElasticNetCV
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

%matplotlib inline

#### Get Data

In [1596]:
ames_train = pd.read_csv('./datasets/outputs/data_cleaning_train_1.0.csv')
ames_test = pd.read_csv('./datasets/outputs/data_cleaning_test_1.0.csv')

In [1597]:
ames_train.drop(columns='Unnamed: 0', inplace=True)
ames_test.drop(columns='Unnamed: 0', inplace=True)

In [1598]:
ames_train.shape, ames_test.shape

((2051, 81), (879, 80))

In [1599]:
data_dict = pd.read_csv('./datasets/outputs/data_dictionary_df.csv')
data_dict.drop(columns='Unnamed: 0', inplace=True)
data_dict

,feature,clean_dt,dataset,description
0,id,int,train,Property number
1,pid,int,train,Row number
2,ms_subclass,int,train,The building class
3,ms_zoning,string,train,Identifies the general zoning classification o...
4,lot_frontage,float,train,Linear feet of street connected to property
...,...,...,...,...
76,misc_val,int,train,$Value of miscellaneous feature
77,mo_sold,int,train,Month Sold
78,yr_sold,int,train,Year Sold
79,sale_type,string,train,Type of sale


In [1600]:
# chek no nulls in train
ames_train.isnull().sum().sort_values(ascending=False).head(5)

saleprice         0
total_bsmt_sf     0
year_remod/add    0
roof_style        0
roof_matl         0
dtype: int64

In [1601]:
# chek no nulls in test
ames_test.isnull().sum().sort_values(ascending=False).head(5)

sale_type         0
yr_sold           0
year_remod/add    0
roof_style        0
roof_matl         0
dtype: int64

---

# One-Hot-Coding

In [1602]:
# Create the mark down column for remaining columns
train_types = ames_train.select_dtypes(include ='object').columns
# print('|Feature|Type|Options|decision|\n|----|----|----|----|')
# for i in train_types:
#     print(f"|{i}|  |  | |")

### Feature Table

Shows the categories and what I should do with columns.

|Feature|Type|Options|decision|drop?|
|----|----|----|----|----|
|ms_subclass| Category | - | Create dummy columns |
|ms_zoning| Category | - | Create dummy columns | |
|street| Category |-  | Create dummy columns | |
|alley|Category  | - |Create dummy columns | |
|lot_shape|Category  |- |Create dummy columns | |
|land_contour|Category  | - |Create dummy columns | |
|utilities| Ordinal | 'AllPub', 'NoSeWa', 'NoSewr' |create ordinal 2,1,0 | |
|lot_config|Category  | - |Create dummy columns | |
|land_slope|Category  | - |Create dummy columns | |
|neighborhood| Category  | - |Create dummy columns | |
|condition_1| Category  | - |Create dummy columns | |
|condition_2| Category  | - |Create dummy columns | |
|bldg_type| Category  | - |Create dummy columns | |
|house_style| Category  | - |Create dummy columns | |
|roof_style| Category  |-  |Create dummy columns |y |
|roof_matl| Category  | - |Create dummy columns |y |
|exterior_1st| Category  | - |Create dummy columns | |
|exterior_2nd| Category  | - |Create dummy columns | |
|mas_vnr_type| Category  | - |Create dummy columns | |
|exter_qual| Ordinal | 'Ex','Gd','TA', 'Fa', 'Po','None'| create ordinal 5,4,3,2,1,0| |
|exter_cond| Ordinal | 'Ex','Gd','TA', 'Fa', 'Po','None'| create ordinal 5,4,3,2,1,0| |
|foundation|  Category  | - |Create dummy columns | |
|bsmt_qual| Ordinal | 'Ex','Gd','TA', 'Fa', 'Po','None'| create ordinal 5,4,3,2,1,0| |
|bsmt_cond| Ordinal | 'Ex','Gd','TA', 'Fa', 'Po','None'| create ordinal 5,4,3,2,1,0| |
|bsmt_exposure| Ordinal |'Gd', 'Av','Mn' 'No', 'None' | create ordinal 3,2,1,0,0| |
|bsmtfin_type_1| Ordinal| 'GLQ','ALQ','BLQ','Rec', 'LwQ' 'Unf','None'| create ordinal 6,5,4,3,2,1,0  | |
|bsmtfin_type_2| Ordinal | 'GLQ','ALQ','BLQ','Rec', 'LwQ' 'Unf','None'| create ordinal 6,5,4,3,2,1,0  | |
|heating|  Category  | - |Create dummy columns | |
|heating_qc| Ordinal |'Ex','Gd','TA', 'Fa', 'Po' | create ordinal 4,3,2,1,0| |
|central_air|Category | - |Create dummy columns | |
|electrical| Category | - |Create dummy columns | |
|kitchen_qual|Ordinal |'Ex','Gd','TA', 'Fa', 'Po' | create ordinal 4,3,2,1,0| |
|functional| Ordinal  | 'Typ', 'Min1','Min2','Mod', 'Maj1', 'Maj2' 'Sev', 'Sal'  |  create ordinal 7,6,5,4,3,2,1,0| | 
|fireplace_qu|Ordinal |'Ex','Gd','TA', 'Fa', 'Po' | create ordinal 4,3,2,1,0| |
|garage_type| Category  | - |Create dummy columns | |
|garage_finish|Ordinal | 'RFn', 'Unf', 'Fin', 'none', 'None' |create ordinal 2,1,0 | |
|garage_qual|  Ordinal | 'Ex','Gd','TA', 'Fa', 'Po','None'| create ordinal 5,4,3,2,1,0| |
|garage_cond|  Ordinal | 'Ex','Gd','TA', 'Fa', 'Po','None','none'| create ordinal 5,4,3,2,1,0,0| |
|paved_drive| Category  | - |Create dummy columns | |
|pool_qc| Ordinal | 'Ex','Gd','TA', 'Fa', 'Po','None'| create ordinal 5,4,3,2,1,0| |
|fence| Ordinal | 'GdPrv','MnPrv', 'GdWo', 'MnWw','None' | create ordinal 4,3,2,1,0||
|misc_feature|  Category  | - |Create dummy columns | |
|sale_type| Category  | - |Create dummy columns |y |

# Observations:

+ Pros for all columns
    + Allow Lasso to decide which columns are important. If they are not then their coefficient will drop to 0
    + Computationally its still a small data set so there is capacity to add them all.
    
+ Con for adding them all
    + May be able to discount some obvious columns
    + Save time

+ I chose to add them all. Especially as this is more thorough and know that the less important ones will be removed.

#### Ordinal Test Columns

In [1603]:
# Need to create 7 dictionaries

# Quality Dictionary
qual_dict = {'Ex':5, "Gd":4, 'TA':3, "Fa":2, "Po":1, "None":0, "none":0}

# Basment fin type Dictionary
bsmt_fin_type_dict = {'GLQ':6, "ALQ":5, 'BLQ':4, "Rec":3, "LwQ":2, "Unf":1, "None":0}

# Basment Exposure Dictionary
bsmnt_exp_dict = {'Gd':4, "Av":3, 'Mn':2, "No":1, "None":0}

# Fence Dictionary
fence_dict = {'GdPrv':4, "MnPrv":3, 'GdWo':2, "MnWw":1, "None":0}

# Garage Finish Dictionary
garage_fn_dict = {'Fin':3,'RFn':3, 'Unf':1,  'none':0, 'None':0 }

# Untilities Dictionary
util_dict = {'AllPub':2, 'NoSeWa':1, 'NoSewr':0 }

# functional Dictionary
functional_dict = {'Typ':7,'Min1':6, "Min2":5, 'Mod':4, 'Maj1':3, "Maj2":2, "Sev":1, "Sal":0}

#### Update ordinal columns

In [1604]:
ames_train['utilities'] = [x if util_dict.get(x)==None else util_dict[x] for x in ames_train['utilities']]

In [1605]:
ames_train['exter_qual'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['exter_qual']]

In [1606]:
ames_train['exter_cond'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['exter_cond']]

In [1607]:
ames_train['bsmt_qual'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['bsmt_qual']]

In [1608]:
ames_train['bsmt_cond'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['bsmt_cond']]

In [1609]:
ames_train['bsmt_exposure'] = [x if bsmnt_exp_dict.get(x)==None else bsmnt_exp_dict[x] for x in ames_train['bsmt_exposure']]

In [1610]:
ames_train['bsmtfin_type_1'] = [x if bsmt_fin_type_dict.get(x)==None 
                                else bsmt_fin_type_dict[x] for x in ames_train['bsmtfin_type_1']]

In [1611]:
ames_train['bsmtfin_type_2'] = [x if bsmt_fin_type_dict.get(x)==None 
                                else bsmt_fin_type_dict[x] for x in ames_train['bsmtfin_type_2']]

In [1612]:
ames_train['heating_qc'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['heating_qc']]

In [1613]:
ames_train['kitchen_qual'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['kitchen_qual']]

In [1614]:
ames_train['functional'] = [x if functional_dict.get(x)==None else functional_dict[x] for x in ames_train['functional']]

In [1615]:
ames_train['fireplace_qu'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['fireplace_qu']]

In [1616]:
ames_train['garage_finish'] = [x if garage_fn_dict.get(x)==None else garage_fn_dict[x] for x in ames_train['garage_finish']]

In [1617]:
ames_train['garage_qual'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['garage_qual']]

In [1618]:
ames_train['garage_cond'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['garage_cond']]

In [1619]:
ames_train['pool_qc'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_train['pool_qc']]

In [1620]:
ames_train['fence'] = [x if fence_dict.get(x)==None else fence_dict[x] for x in ames_train['fence']]

---

#### Train Dummies

In [1621]:
ames_train =  pd.get_dummies(ames_train, columns=['ms_subclass'], drop_first=True, prefix="ms_sub")

In [1622]:
ames_train =  pd.get_dummies(ames_train, columns=['ms_zoning'], drop_first=True, prefix="msz")

In [1623]:
ames_train = pd.get_dummies(ames_train, columns=['street'], prefix="str")

In [1624]:
ames_train = pd.get_dummies(ames_train, columns=['alley'], prefix="aly")

In [1625]:
ames_train = pd.get_dummies(ames_train, columns=['lot_shape'], prefix="lts")

In [1626]:
ames_train = pd.get_dummies(ames_train, columns=['land_contour'], prefix="ld_ct")

In [1627]:
ames_train = pd.get_dummies(ames_train, columns=['lot_config'], prefix="lt_cfg")

In [1628]:
ames_train = pd.get_dummies(ames_train, columns=['land_slope'], prefix="ld_sl")

In [1629]:
ames_train = pd.get_dummies(ames_train, columns=['condition_1'], prefix="cd1")

In [1630]:
ames_train = pd.get_dummies(ames_train, columns=['condition_2'], prefix="cd2")

In [1631]:
ames_train = pd.get_dummies(ames_train, columns=['bldg_type'], prefix="bd_ty")

In [1632]:
ames_train = pd.get_dummies(ames_train, columns=['house_style'], prefix="hs_sty")

In [1633]:
#ames_train = pd.get_dummies(ames_train, columns=['roof_style'], prefix="rf_sty")

In [1634]:
#ames_train = pd.get_dummies(ames_train, columns=['roof_matl'], prefix="rf_mat")

In [1635]:
ames_train = pd.get_dummies(ames_train, columns=['exterior_1st'], prefix="ext1")

In [1636]:
ames_train = pd.get_dummies(ames_train, columns=['exterior_2nd'], prefix="ext2")

In [1637]:
ames_train = pd.get_dummies(ames_train, columns=['mas_vnr_type'], prefix="mvt")

In [1638]:
ames_train = pd.get_dummies(ames_train, columns=['foundation'], prefix="fd")

In [1639]:
ames_train = pd.get_dummies(ames_train, columns=['heating'], prefix="ht")

In [1640]:
ames_train = pd.get_dummies(ames_train, columns=['central_air'], prefix="ac")

In [1641]:
ames_train = pd.get_dummies(ames_train, columns=['electrical'], prefix="elc")

In [1642]:
ames_train = pd.get_dummies(ames_train, columns=['garage_type'], prefix="gtyp")

In [1643]:
#ames_train = pd.get_dummies(ames_train, columns=['paved_drive'], prefix="pav")

In [1644]:
#ames_train = pd.get_dummies(ames_train, columns=['sale_type'], prefix="styp")

In [1645]:
ames_train = pd.get_dummies(ames_train, columns=['neighborhood'], prefix="nbd_")

In [1646]:
#ames_train =  pd.get_dummies(ames_train, columns=['misc_feature'], prefix="mf_")

---

### Test data

In [1647]:
# Update ordinal columns

In [1648]:
ames_test['utilities'] = [x if util_dict.get(x)==None else util_dict[x] for x in ames_test['utilities']]

In [1649]:
ames_test['exter_qual'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['exter_qual']]

In [1650]:
ames_test['exter_cond'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['exter_cond']]

In [1651]:
ames_test['bsmt_qual'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['bsmt_qual']]

In [1652]:
ames_test['bsmt_cond'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['bsmt_cond']]

In [1653]:
ames_test['bsmt_exposure'] = [x if bsmnt_exp_dict.get(x)==None 
                              else bsmnt_exp_dict[x] for x in ames_test['bsmt_exposure']]

In [1654]:
ames_test['bsmtfin_type_1'] = [x if bsmt_fin_type_dict.get(x)==None 
                               else bsmt_fin_type_dict[x] for x in ames_test['bsmtfin_type_1']]

In [1655]:
ames_test['bsmtfin_type_2'] = [x if bsmt_fin_type_dict.get(x)==None 
                               else bsmt_fin_type_dict[x] for x in ames_test['bsmtfin_type_2']]

In [1656]:
ames_test['bsmtfin_type_2'].unique()

array([1, 2, 0, 5, 6, 3, 4])

In [1657]:
ames_test['heating_qc'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['heating_qc']]

In [1658]:
ames_test['kitchen_qual'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['kitchen_qual']]

In [1659]:
ames_test['functional'] = [x if functional_dict.get(x)==None else functional_dict[x] for x in ames_test['functional']]

In [1660]:
ames_test['fireplace_qu'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['fireplace_qu']]

In [1661]:
ames_test['garage_finish'] = [x if garage_fn_dict.get(x)==None else garage_fn_dict[x] for x in ames_test['garage_finish']]

In [1662]:
ames_test['garage_qual'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['garage_qual']]

In [1663]:
ames_test['garage_cond'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['garage_cond']]

In [1664]:
ames_test['pool_qc'] = [x if qual_dict.get(x)==None else qual_dict[x] for x in ames_test['pool_qc']]

In [1665]:
ames_test['fence'] = [x if fence_dict.get(x)==None else fence_dict[x] for x in ames_test['fence']]

In [1666]:
# Train Dummies

In [1667]:
ames_test =  pd.get_dummies(ames_test, columns=['ms_subclass'], drop_first=True, prefix="ms_sub")

In [1668]:
ames_test =  pd.get_dummies(ames_test, columns=['ms_zoning'], drop_first=True, prefix="msz")

In [1669]:
ames_test = pd.get_dummies(ames_test, columns=['street'], prefix="str")

In [1670]:
ames_test = pd.get_dummies(ames_test, columns=['alley'], prefix="aly")

In [1671]:
ames_test = pd.get_dummies(ames_test, columns=['lot_shape'], prefix="lts")

In [1672]:
ames_test = pd.get_dummies(ames_test, columns=['land_contour'], prefix="ld_ct")

In [1673]:
ames_test = pd.get_dummies(ames_test, columns=['lot_config'], prefix="lt_cfg")

In [1674]:
ames_test = pd.get_dummies(ames_test, columns=['land_slope'], prefix="ld_sl")

In [1675]:
ames_test = pd.get_dummies(ames_test, columns=['condition_1'], prefix="cd1")

In [1676]:
ames_test = pd.get_dummies(ames_test, columns=['condition_2'], prefix="cd2")

In [1677]:
ames_test = pd.get_dummies(ames_test, columns=['bldg_type'], prefix="bd_ty")

In [1678]:
ames_test = pd.get_dummies(ames_test, columns=['house_style'], prefix="hs_sty")

In [1679]:
#ames_test = pd.get_dummies(ames_test, columns=['roof_style'], prefix="rf_sty")

In [1680]:
#ames_test = pd.get_dummies(ames_test, columns=['roof_matl'], prefix="rf_mat")

In [1681]:
ames_test = pd.get_dummies(ames_test, columns=['exterior_1st'], prefix="ext1")

In [1682]:
ames_test = pd.get_dummies(ames_test, columns=['exterior_2nd'], prefix="ext2")

In [1683]:
ames_test = pd.get_dummies(ames_test, columns=['mas_vnr_type'], prefix="mvt")

In [1684]:
ames_test = pd.get_dummies(ames_test, columns=['foundation'], prefix="fd")

In [1685]:
ames_test = pd.get_dummies(ames_test, columns=['heating'], prefix="ht")

In [1686]:
ames_test = pd.get_dummies(ames_test, columns=['central_air'], prefix="ac")

In [1687]:
ames_test = pd.get_dummies(ames_test, columns=['electrical'], prefix="elc")

In [1688]:
ames_test = pd.get_dummies(ames_test, columns=['garage_type'], prefix="gtyp")

In [1689]:
#ames_test = pd.get_dummies(ames_test, columns=['paved_drive'], prefix="pav")

In [1690]:
#ames_test = pd.get_dummies(ames_test, columns=['sale_type'], prefix="styp")

In [1691]:
ames_test = pd.get_dummies(ames_test, columns=['neighborhood'], prefix="nbd_")

In [1692]:
#ames_test =  pd.get_dummies(ames_test, columns=['misc_feature'], prefix="mf_")

#### Check cols
Some data in the test has more category items in the columns. Therefore I need to check that the columns are present in both the test data and the train data 

In [1693]:
# Check the train data isnt missing any columns 

missing_cols_test = []
for i in ames_train.columns:
    if i not in ames_test.columns:
        missing_cols_test.append(i)
        
missing_cols_test

['saleprice',
 'ms_sub_150',
 'msz_C (all)',
 'cd2_Artery',
 'cd2_RRAe',
 'cd2_RRAn',
 'cd2_RRNn',
 'ext1_CBlock',
 'ext1_ImStucc',
 'ext1_Stone',
 'ext2_Stone',
 'ht_OthW',
 'ht_Wall',
 'elc_Mix',
 'gtyp_none',
 'nbd__GrnHill',
 'nbd__Landmrk']

In [1694]:
# Add data to the missing columns

for i in missing_cols_test:
    ames_test[i] = 0

In [1695]:
# Check the train data isnt missing any columns 

missing_cols_train = []
for i in ames_test.columns:
    if i not in ames_train.columns:
        missing_cols_train.append(i)
        
missing_cols_train

['ext1_PreCast', 'ext2_Other', 'ext2_PreCast', 'mvt_CBlock', 'ht_Floor']

In [1696]:
# Add data to the missing columns

for i in missing_cols_train:
    ames_train[i] = 0

In [1697]:
# remove the sale price from test as it shouldnt be there

ames_test.drop(columns='saleprice', inplace=True)

---

## Save the Data

In [1698]:
ames_train.to_csv('./datasets/outputs/final_train.csv', index=False)

In [1699]:
ames_test.to_csv('./datasets/outputs/final_test.csv', index=False)